In [1]:
import cv2
import open3d as o3d
import numpy as np
import colorsys
import pyransac3d as pyrsc

Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


In [2]:
def visualizer(cloud):
    o3d.visualization.draw_geometries([cloud],
                                  zoom=0.5,
                                  front=[1, 0, 0],
                                  lookat=[0, 0, 0],
                                  up=[0, 0, 1])


In [98]:
def display_inlier_outlier(cloud, ind):
    inlier_cloud = cloud.select_by_index(ind)
    outlier_cloud = cloud.select_by_index(ind, invert=True)

    print("Showing outliers (red) and inliers (gray): ")
    # outlier_cloud.paint_uniform_color([1, 0, 0])
    # inlier_cloud.paint_uniform_color([0.8, 0.8, 0.8])
    o3d.visualization.draw_geometries([inlier_cloud],
                                       zoom=0.5,
                                  front=[1, 0, 0],
                                  lookat=[0, 0, 0],
                                  up=[0, 0, 1])

pcd = o3d.io.read_point_cloud("3dmodels/Strawberry_2_Gaussian_Splat.ply")

print(pcd)


# o3d.visualization.draw_geometries([downpcd],
#                                   zoom=0.5,
#                                   front=[1, 0, 0],
#                                   lookat=[0, 0, 0],
#                                   up=[0, 0, 1])

# visualizer(pcd)
downpcd = pcd

# display_inlier_outlier(pcd, ind)

PointCloud with 1181860 points.


In [5]:
# visualizer(downpcd)

In [6]:
all_rgb_colors = np.asarray(downpcd.colors)

In [7]:
downpcd.colors[0]

array([0.18431373, 0.20392157, 0.18039216])

In [8]:
all_hsv_colors = []

for i in range(len(all_rgb_colors)):
    temp_color = all_rgb_colors[i]
    temp = colorsys.rgb_to_hsv(temp_color[0], temp_color[1], temp_color[2])
    all_hsv_colors.append(temp)

all_hsv_colors = np.asarray(all_hsv_colors)

In [9]:
def hsv_filter(hsv_color):
    low_h = 37/360
    high_h = 180/360

    low_s = 14/255
    high_s = 255/255

    low_v = 141/255
    high_v = 255/255

    # red
    # Lower bound: (0, 100, 100)
    # Upper bound: (10, 255, 255)

    if low_h <= hsv_color[0] <= high_h and low_s <= hsv_color[1] <= high_s and low_v <= hsv_color[2] <= high_v:
        return True
    else:
        return False
    

In [10]:
def hsv_filter_red(hsv_color):
    # low_h = 0/180
    # high_h = 180/180

    # low_s = 100/255
    # high_s = 255/255

    # low_v = 100/255
    # high_v = 255/255

    # # red
    # # Lower bound: (0, 100, 100)
    # # Upper bound: (10, 255, 255)

    # if low_h <= hsv_color[0] <= high_h and low_s <= hsv_color[1] <= high_s and low_v <= hsv_color[2] <= high_v:
    #     return True
    # else:
    #     return False

    # print(red)


    if red >= (150/255) and red <= (250/255):
        return True
    else:
        return False
    

In [11]:
new_points = []
new_colors = []
new_colors_RGB_u = []

for index, new_color in enumerate(all_hsv_colors):
    new_point = downpcd.points[index]
    new_color_RGB = all_rgb_colors[index]
    if hsv_filter(new_color):
        new_colors_RGB_u.append(new_color_RGB)
        new_points.append(new_point)
        new_colors.append(new_color)
        # print(new_color_RGB)
    elif hsv_filter_red(all_rgb_colors[index]):
        new_colors_RGB_u.append(new_color_RGB)
        new_points.append(new_point)
        new_colors.append(new_color)
        # print("yes")

new_colors_RGB_u = np.asarray(new_colors_RGB_u)
new_colors = np.asarray(new_colors)
new_points = np.asarray(new_points)

In [12]:
pcd_hsv = o3d.geometry.PointCloud()
pcd_hsv.points = o3d.utility.Vector3dVector(new_points)
pcd_hsv.colors = o3d.utility.Vector3dVector(new_colors_RGB_u)
visualizer(pcd_hsv)

In [13]:
cl, ind = pcd_hsv.remove_radius_outlier(nb_points=50, radius=0.03)

visualizer(cl)

In [14]:
cl, ind = cl.remove_statistical_outlier(nb_neighbors=70,
                                                    std_ratio=1.0)
cl.estimate_normals()
visualizer(cl)
# save the point cloud
# o3d.io.write_point_cloud("saved_point_cloud.ply", cl)

[Open3D WARNING] GLFW Error: WGL: Failed to make context current: The requested transformation operation is not supported. 


In [229]:
s = 1  #no. of samples used every time to create the shape
p = 0.99 #99%
e = 0.5 # 50%
k = np.log(1-p)/(np.log(1 - np.power((1-e), s)))
print(k)
# k => number of iterations

6.643856189774724


In [108]:

# points = np.load_points(cl) # Load your point cloud as a numpy array (N, 3)

plane1 = pyrsc.Plane()
# s = 3
best_eq, best_inliers = plane1.fit(np.asarray(downpcd.points), thresh=0.1)

# plane1 = pyrsc.Plane()
# # s = 3
# best_eq_plane, best_inliers_plane = plane1.fit(np.asarray(inlier_cloud_point.points), thresh=0.15, maxIteration=40)

# inlier_cloud_plane = inlier_cloud_point.select_by_index(best_inliers_plane)
# outlier_cloud_plane = inlier_cloud_point.select_by_index(best_inliers_plane, invert=True)

# outlier_cloud_plane.paint_uniform_color([1, 0, 0])
# inlier_cloud_plane.paint_uniform_color([0, 1, 0])
# o3d.visualization.draw_geometries([inlier_cloud_plane, outlier_cloud_plane],
#                                     zoom=0.5,
#                                     front=[1, 0, 0],
#                                     lookat=[0, 0, 0],
#                                     up=[0, 0, 1])


In [128]:
outlier_cloud_plane = downpcd.select_by_index(best_inliers, invert=True)
print(len(outlier_cloud_plane.points))

309035


In [143]:
sphere = pyrsc.Sphere()
# s = 4
center, radius, best_inliers_sphere = sphere.fit(np.asarray(downpcd.points), thresh=0.1, maxIteration=75)  #better

In [208]:
print(best_inliers_sphere)

[      0       1       2 ... 1181857 1181858 1181859]


In [144]:
inlier_cloud = downpcd.select_by_index(best_inliers_sphere)
outlier_cloud = downpcd.select_by_index(best_inliers_sphere, invert=True)

print("Showing outliers (red) and inliers (gray): ")
# outlier_cloud.paint_uniform_color([1, 0, 0])
inlier_cloud.paint_uniform_color([0, 1, 0])
o3d.visualization.draw_geometries([outlier_cloud],
                                    zoom=0.5,
                                front=[1, 0, 0],
                                lookat=[0, 0, 0],
                                up=[0, 0, 1])

Showing outliers (red) and inliers (gray): 


In [227]:
new_c = downpcd.select_by_index(best_inliers_sphere, invert=True)
cenetr = new_c.get_center()
single_point_cloud = o3d.geometry.PointCloud()
point = np.array([0, 0, 0])  # Replace x, y, z with your desired coordinates

single_point_cloud.points = o3d.utility.Vector3dVector(np.asarray([point]))
single_point_cloud.paint_uniform_color([1, 0, 0])


PointCloud with 1 points.

In [228]:
# downpcd.estimate_normals()
# new = downpcd.detect_planar_patches()
o3d.visualization.draw_geometries([new_c, single_point_cloud])
# open3d.ml.torch.layers.RadiusSearch

In [224]:
def squared_distances_3d(center, radius, points):

    print("before", len(points))

    x, y, z = center
    np_points = np.asarray(points)
    distances_squared = np.sum((np_points - [x, y, z])**2, axis=1)

    mask = distances_squared <= radius**2

    filtered_points = np_points[mask]

    filtered_indexes = np.argwhere(mask == True)

    print("after", len(filtered_points))

    return (filtered_points, filtered_indexes)


(new_points, new_indexes) = squared_distances_3d((0, 0, 0), 10, new_c.points)


before 51461
after 51461


In [225]:
# filtered_point_cloud = o3d.geometry.PointCloud()
# filtered_point_cloud.points = o3d.utility.Vector3dVector(new_points)
new_c = downpcd.select_by_index(new_indexes)
visualizer(new_c)


C:\Users\vmuriki3\AppData\Local\Temp\ipykernel_11724\863972166.py:3: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  new_c = downpcd.select_by_index(new_indexes)


In [199]:
radii = [0.005, 0.01, 0.02, 0.04]
rec_mesh = o3d.geometry.TriangleMesh.create_from_point_cloud_ball_pivoting(
    cl, o3d.utility.DoubleVector(radii))
# o3d.io.write_triangle_mesh("saved_mesh.obj", rec_mesh)

[Open3D WARNING] Write OBJ can not include triangle normals.


True

In [ ]:
# Ransac
# When data is affected by outliers, RANSAC removes outliers
# - trial and error approach - inlier set and an outlier set
# repeat until max steps and select the model which has the highest score
# - sample set of points - inliers assume
# - compute model params
# - scoring step - how many of the remaining points support this model

In [ ]:
box = mesh.get_oriented_bounding_box()
# visualizer(mesh)
vertices = np.asarray(box.get_box_points())

# Define the edges of the oriented bounding box
lines = [
    [0, 1], [1, 2], [2, 3], [3, 0],
    [4, 5], [5, 6], [6, 7], [7, 4],
    [0, 4], [1, 5], [2, 6], [3, 7]
]

# Create a LineSet to represent the box
line_set = o3d.geometry.LineSet(
    points=o3d.utility.Vector3dVector(vertices),
    lines=o3d.utility.Vector2iVector(lines)
)

# Set the color of the box (e.g., red)
line_set.colors = o3d.utility.Vector3dVector(np.tile([1, 0, 0], (len(lines), 1)))
o3d.visualization.draw_geometries([mesh, line_set],
                                zoom=0.5,
                                front=[1, 0, 0],
                                lookat=[-0.05272632, -0.02440209, 0.48188474],
                                up=[0, 0, 1])